# Objectives
In this Notebook, we will apply image augmentation to the dataset from the previous exercise hopefully we can allegiate overfitting.

After going through this notebook, you will be able to:
- Apply image augmentation

Requirments:
- Data used in this notebook are downloaded in the previous notebook.
- Install are mentionned in the exercise 1 as well.

# Requirements

This notebook should use torchvision and torch versions that are compatible with each other. For more information, visit [this link](https://pypi.org/project/torchvision/).

For demonstration purposes, we use the torch version 1.12.1 and the torchvision 0.13.1. However, users may install and use other versions.

In [2]:
#Env: azureml_py38_PT_TF
import torch
import torchvision
print('The version of PyTorch is {}'.format(torch.__version__))
print('The version of Torchvision is {}'.format(torchvision.__version__))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('There are {} cuda devices'.format(torch.cuda.device_count()))
print('Used device is {}'.format(device))

The version of PyTorch is 1.12.1
The version of Torchvision is 0.13.1
There are 1 cuda devices
Used device is cuda


# Common imports
The below improts are encountered in most of the computer vision projects. Though, you may need to adapt them according to your needs.

In [3]:
#Imports
import json, os, random
random.seed(1985)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import random


from tqdm import tqdm
from matplotlib.patches import Rectangle
from PIL import Image


from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset

from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.autograd import Variable
from torchvision.ops import nms


from torch_snippets import *
from project_utils import *


# Configuring some pathes to make it easier to call images and annotations

In [6]:
#The pathes on AzureML are a bit strange
full_path = os.getcwd().split('/')
#This will adapt the user name to your specific machine
home = os.path.join(os.path.expanduser('~'), 'cloudfiles/code/Users/' + full_path[11])
print('Home Directory - Full Path: {}\n'.format(home))

#The Notebook directory
notebook_home = os.path.join(home, 'computer_vision_with_azure_course/1_object_detection')
print('Notebbok Directory: {}\n'.format(notebook_home))

#The Notebook directory
images_home = os.path.join(home, 'data/rumex/images')
print('Images Directory: {}\n'.format(images_home))

#The Notebook directory
annotations_home = os.path.join(home, 'computer_vision_with_azure_course/1_object_detection/annotations')
print('Annotations Directory: {}\n'.format(annotations_home))

Home Directory - Full Path: /home/azureuser/cloudfiles/code/Users/roland.nasser.agroscope

Notebbok Directory: 
/home/azureuser/cloudfiles/code/Users/roland.nasser.agroscope/computer_vision_with_azure_course/1_object_detection

Images Directory: /home/azureuser/cloudfiles/code/Users/roland.nasser.agroscope/data/rumex/images

Annotations Directory: 
/home/azureuser/cloudfiles/code/Users/roland.nasser.agroscope/computer_vision_with_azure_course/1_object_detection/
annotations

# Loading annotations


In [7]:
#Test the function
annotations_link = os.path.join(annotations_home, 'annotations.csv')
df = load_clean_annotations(annotations_link, images_home)
df.head()

,Label,External ID
0,"{""Wurzelpunkt"":[{""geometry"":{""x"":821,""y"":424}}...",img_2001.png
1,"{""Wurzelpunkt"":[{""geometry"":{""x"":651,""y"":493}}...",img_2008.png
2,"{""Wurzelpunkt"":[{""geometry"":{""x"":545,""y"":699}}...",img_2009.png
3,"{""Wurzelpunkt"":[{""geometry"":{""x"":740,""y"":527}}...",img_2010.png
4,"{""Wurzelpunkt"":[{""geometry"":{""x"":697,""y"":666}}...",img_2011.png


# Image augmentation

In [ ]:
import transforms as T

def get_transform(train):
    transforms = []
    #transforms.append(T.PILToTensor())
    #transforms.append(T.ConvertImageDtype(torch.float))
    #if train:
        #transforms.append(T.RandomHorizontalFlip(0.5))
        #transforms.append(T.RandomIoUCrop())
        #transforms.append(T.RandomPhotometricDistort())
    return T.Compose(transforms)


In [ ]:
from sklearn.model_selection import train_test_split

annotations_link = os.path.join(annotations_home, 'annotations.csv')
annotations_df = load_clean_annotations(annotations_link, images_home)


trn_ids, val_ids = train_test_split(df['External ID'].unique(), test_size=0.3, random_state=99)
trn_df, val_df = df[df['External ID'].isin(trn_ids)], df[df['External ID'].isin(val_ids)]

trn_df.reset_index(inplace = True, drop=True)
val_df.reset_index(inplace = True, drop=True)

print(len(trn_df))
print(len(val_df))

In [ ]:
train_data = RumexDataSetLabelBox(images_home, trn_df, transforms=get_transform('train'))
val_data = RumexDataSetLabelBox(images_home, val_df)


In [ ]:
train_loader = DataLoader(
        train_data,
        batch_size=8,
        shuffle=False,
        collate_fn=collate_fn
    )

val_loader = DataLoader(
        val_data,
        batch_size=8,
        shuffle=False,
        collate_fn=collate_fn
    )

In [ ]:
print('Make sure the device is a cuda device: \n Found device: {}'.format(device))
model = get_model().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.05)
n_epochs = 20
log = Report(n_epochs) #From pytorch_snipets


In [ ]:
with torch.no_grad():
    for epoch in range(n_epochs):
        _n = len(train_loader)
        for ix, inputs in enumerate(train_loader):
            loss, losses = train_batch(inputs, model, optimizer)
            loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = \
                [losses[k] for k in ['loss_classifier','loss_box_reg','loss_objectness','loss_rpn_box_reg']]
            pos = (epoch + (ix+1)/_n)
            
            log.record(pos, trn_loss=loss.item(), trn_loc_loss=loc_loss.item(), 
                    trn_regr_loss=regr_loss.item(), trn_objectness_loss=loss_objectness.item(),
                    trn_rpn_box_reg_loss=loss_rpn_box_reg.item(), end='\r')
            

        _n = len(val_loader)
        for ix,inputs in enumerate(val_loader):
            loss, losses = validate_batch(inputs, model, optimizer)
            loc_loss, regr_loss, loss_objectness, loss_rpn_box_reg = \
            [losses[k] for k in ['loss_classifier','loss_box_reg','loss_objectness','loss_rpn_box_reg']]
            pos = (epoch + (ix+1)/_n)
            
            log.record(pos, val_loss=loss.item(), val_loc_loss=loc_loss.item(), 
                    val_regr_loss=regr_loss.item(), val_objectness_loss=loss_objectness.item(),
                    val_rpn_box_reg_loss=loss_rpn_box_reg.item(), end='\r')
            
        if (epoch+1)%(n_epochs//5)==0:
            #call("nvidia-smi")
            log.report_avgs(epoch+1)

In [ ]:
log.plot_epochs(['trn_loss','val_loss'])

In [ ]:
model.eval()
for ix, (images, targets) in enumerate(test_loader):
    if ix==1: break
    images = [im for im in images]
    outputs = model(images)
    
    for ix, output in enumerate(outputs):
        bbs, confs, labels = decode_output(output)
        #print(bbs)
        #print(confs)
        #print(labels)
        #info = [f'{l}@{c:.2f}' for l,c in zip(labels, confs)]
        show(images[ix].cpu().permute(1,2,0), bbs=bbs, texts=labels, sz=5) #Show from pytorch snippets

In [ ]:
torch.save(model.state_dict(), "models/model_augmentation.pth")
print("Saved PyTorch Model State to model.pth")